# 감성분석 모델 구축

In [ ]:
# ! pip install scikit-learn
# ! pip install konlpy

In [ ]:
import re
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
# from konlpy.tag import Mecab
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

warnings.filterwarnings(action = 'ignore') # warning 메시지 표시x

### 2. 훈련모델Ⅱ - 200,000개 + 14,060개
- 네이버 쇼핑 리뷰로 훈련(train)
- 쿠팡리뷰로 평가(test)

In [14]:
naver_for_train = pd.read_excel('(0)naver_reviews.xlsx')
coupang_for_test = pd.read_excel('(3)coupang_reviews.xlsx')

# 0. 훈련 데이터와 테스트 데이터를 3:1 비율로 분리
print('훈련용 리뷰의 개수 :', len(naver_for_train))
print('테스트용 리뷰의 개수 :', len(coupang_for_test))
# 훈련용 리뷰의 개수 : 200,000개
# 테스트용 리뷰의 개수 : 14,060

# 1. 라벨 분포 확인
naver_for_train['label'].value_counts() # 0: 100037 & 1: 99963
coupang_for_test['label'].value_counts() # 1: 13489 & 0: 571

# 2. 한글 외의 문자 제거
naver_for_train['reviews'] = naver_for_train['reviews'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
coupang_for_test['reviews'] = coupang_for_test['reviews'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))

# naver_for_train.head()
# coupang_for_test.head()

훈련용 리뷰의 개수 : 200000
테스트용 리뷰의 개수 : 14060


,ratings,reviews,label
0,5,가성비 갑 성능 좋아요 추천드립니다 내돈내산 후기입니다 구매 및 개봉일자 ...,1
1,5,전직 컴수리기사의 버즈 테스트 결과보고서 주 문 도 착 ...,1
2,5,가성비 그자체의 무선 이어폰입니다 내돈내산 솔직후기 구매일 두달가량 사용해보고...,1
3,5,러닝용으로 구매했는데 절대 안빠지고 잘 버텨주는 제품 작년 겨울에 쿠팡에서 산 저렴...,1
4,5,화이트 갤럭시 버즈 내돈내산 최고 삼성전자 갤럭시 버즈 블루투스 이어...,1


In [15]:
# 3. 분석 모델 구축 전 작업

# 1)형태소 분석 - 문장을 토큰화
okt = Okt()
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

# 2)TF-IDF 벡터화에 사용할 tfidf 객체 생성 -> 벡터로 변환(transform)
tfidf = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf.fit(naver_for_train['reviews'])
model2_train_tfidf = tfidf.transform(naver_for_train['reviews'])
model2_train_tfidf

# 19분

<200000x158594 sparse matrix of type '<class 'numpy.float64'>'
	with 4231021 stored elements in Compressed Sparse Row format>

In [16]:
# 4. 분석 모델 구축

SA_lr = LogisticRegression(random_state=0)
SA_lr.fit(model2_train_tfidf, naver_for_train['label'])

params = {'C': [1,3,3.5,4,4.5,5]}
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

SA_lr_grid_cv.fit(model2_train_tfidf, naver_for_train['label'])

print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

SA_lr_best2 = SA_lr_grid_cv.best_estimator_ 

# {'C': 3.5} 0.9153

Fitting 3 folds for each of 6 candidates, totalling 18 fits
{'C': 3.5} 0.9153


In [17]:
# 5. 분석 모델 평가1 - 평가용 데이터를 이용하여 모델 정확도 확인하기

model2_test_tfidf = tfidf.transform(coupang_for_test['reviews'])

test_predict = SA_lr_best2.predict(model2_test_tfidf)

# 평가용 데이터의 감성 결과값nsmc_test_df['label']과 감성 예측값test_predict을 기반으로 정확도를 계산accuracy_score( )하여출력

print('감성 분석 모델의 정확도 : ', round(accuracy_score(coupang_for_test['label'], test_predict), 3))
# 감성 분류 모델의 정확도가 65.2%!
# * -> 각종 상품군으로 훈련시키고, 정작 평가는 무선 이어폰으로만 진행했으니, 모델 정확도가 낮아짐
    # 선호하는 키워드들이 다르므로

감성 분석 정확도 :  0.652


### => 감성 분석 정확도가 낮게 나왔으므로,
- 정확한 원인에 관한 시각화 분석 진행

In [ ]:

# todo: naver와 coupang의 각각의 키워드 빈도 분석 및 시각화 ㄱㄱ
# todo: 딴 애긴데, 정확도 제일 높은걸로 예측모델 만들기 - https://wikidocs.net/94600 여기있는걸로
    # xx퍼 확률로 긍정,부정 이렇게 나뉘는거 ㄷㄷ